# Проект по SQL

## **Описание проекта**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Моя первая задача как аналитика — проанализировать базу данных.

**Цели проекта:** проанализировать базу данных. В базе данных информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Задачи**

* Посчитать сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитать количество обзоров и среднюю оценку;
* Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, чтобы исключить из анализа брошюры;
* Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
* Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## **Описание данных**

**Таблица books** - cодержит данные о книгах:
* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.

**Таблица authors** - cодержит данные об авторах:
* author_id — идентификатор автора;
* author — имя автора.

**Таблица publishers** - cодержит данные об издательствах:

* publisher_id — идентификатор издательства;
* publisher — название издательства;

**Таблица ratings** - cодержит данные о пользовательских оценках книг:
* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.

**Таблица reviews** - содержит данные о пользовательских обзорах на книги:
* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя пользователя, написавшего обзор;
* text — текст обзора.

## **Подключение к БД и просмотр таблиц**

In [1]:
# Импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

# Устанавливаем параметры
db_config = {'user': 'praktikum_student', # Имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # Пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # Порт подключения
 'db': 'data-analyst-final-project-db'} # Название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# Сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

**С помощью pd.set_option настроим вывод так, чтобы текст в ячейке отражался полностью, вне зависимости от длины, все числа отображались с двумя знаками после запятой**

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', '{:.2f}'.format) 

**Таблица books cодержит данные о книгах**

In [3]:
# Формируем sql-запрос. Выполняем запрос и сохраняем результат, выполнения в DataFrame
query1 = ''' SELECT *
             FROM books
         '''
# Sqlalchemy автоматически установит названия колонок, такими же, как у таблицы в БД
books = pd.io.sql.read_sql(query1, con = engine)
books['publication_date'] = pd.to_datetime(books['publication_date'])

# Код для выгрузки данных из таблицы books в файл books.csv
#books.to_csv('books.csv')

display(books.info()) # получение общей информации
books.head(10) # получение первых 10 строк таблицы

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   book_id           1000 non-null   int64         
 1   author_id         1000 non-null   int64         
 2   title             1000 non-null   object        
 3   num_pages         1000 non-null   int64         
 4   publication_date  1000 non-null   datetime64[ns]
 5   publisher_id      1000 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 47.0+ KB


None

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


**Таблица authors cодержит данные об авторах**

In [4]:
# Формируем sql-запрос. Выполняем запрос и сохраняем результат, выполнения в DataFrame
query2 = ''' SELECT *
             FROM authors
         '''
# Sqlalchemy автоматически установит названия колонок, такими же, как у таблицы в БД
authors = pd.io.sql.read_sql(query2, con = engine)

# Код для выгрузки данных из таблицы authors в файл authors.csv
#authors.to_csv('authors.csv')

display(authors.info()) # получение общей информации
authors.head(10) # получение первых 10 строк таблицы

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


None

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


**Таблица publishers cодержит данные об издательствах**

In [5]:
# Формируем sql-запрос. Выполняем запрос и сохраняем результат, выполнения в DataFrame
query3 = ''' SELECT *
             FROM publishers
         '''
# Sqlalchemy автоматически установит названия колонок, такими же, как у таблицы в БД
publishers = pd.io.sql.read_sql(query3, con = engine)

# Код для выгрузки данных из таблицы publishers в файл publishers.csv
#publishers.to_csv('publishers.csv')

display(publishers.info()) # получение общей информации
publishers.head(10) # получение первых 10 строк таблицы

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


None

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


**Таблица ratings cодержит данные о пользовательских оценках книг**

In [6]:
# Формируем sql-запрос. Выполняем запрос и сохраняем результат, выполнения в DataFrame
query4 = ''' SELECT *
             FROM ratings
         '''
# Sqlalchemy автоматически установит названия колонок, такими же, как у таблицы в БД
ratings = pd.io.sql.read_sql(query4, con = engine)

# Код для выгрузки данных из таблицы ratings в файл ratings.csv
#ratings.to_csv('ratings.csv')

display(ratings.info()) # получение общей информации
ratings.head(10) # получение первых 10 строк таблицы

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


**Таблица reviews cодержит данные о пользовательских обзорах на книги**

In [7]:
# Формируем sql-запрос. Выполняем запрос и сохраняем результат, выполнения в DataFrame
query5 = ''' SELECT *
             FROM reviews
         '''
# Sqlalchemy автоматически установит названия колонок, такими же, как у таблицы в БД
reviews = pd.io.sql.read_sql(query5, con = engine)

# Код для выгрузки данных из таблицы reviews в файл reviews.csv
#reviews.to_csv('reviews.csv')

display(reviews.info()) # получение общей информации
reviews.head(10) # получение первых 10 строк таблицы

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With personal where occur direction foot. Half sit role want.
7,8,4,abbottjames,Within enough mother. There at system full recent play recognize.
8,9,5,npowers,Thank now focus realize economy focus fly. Item step fine war. Western service history anything.
9,10,5,staylor,Game push lot reduce where remember. Including song travel hair foot. Director always myself past story chance reason.


**Вывод**
* У нас имеется к анализу 1000 книг 636 авторов. 
* Книги выпущены в 340 издательствах.
* Для книг также есть пользовательские оценки (6456 оценок) и отзывы (2793 обзоров). 

Теперь можно приступать к ответам на вопросы.

## Ответы на вопросы

In [8]:
# Напишем функцию для выполнения запросов.
def display(query):
    return pd.io.sql.read_sql(query, con = engine)

### Посчитаем сколько книг вышло после 1 января 2000 года

In [9]:
cnt_books = '''
SELECT
    COUNT(*) AS cnt_books
FROM
    books
WHERE
    publication_date > '2000-01-01';
'''
display(cnt_books)

,cnt_books
0,819


819 книг вышло после 1 января 2000 года.

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [10]:
ratings_reviews = '''
SELECT 
    books.title AS title,
    books.book_id AS id,
    COUNT(DISTINCT reviews.text) AS cnt_reviews,
    AVG(ratings.rating) AS avg_rating
FROM
    books
LEFT JOIN reviews ON reviews.book_id = books.book_id
LEFT JOIN ratings ON ratings.book_id = books.book_id

GROUP BY
     books.title,
     books.book_id
'''

display(ratings_reviews)

,title,id,cnt_reviews,avg_rating
0,'Salem's Lot,1,2,3.67
1,1 000 Places to See Before You Die,2,1,2.50
2,13 Little Blue Envelopes (Little Blue Envelope #1),3,3,4.67
3,1491: New Revelations of the Americas Before Columbus,4,2,4.50
4,1776,5,4,4.00
...,...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),996,3,3.67
996,Xenocide (Ender's Saga #3),997,3,3.40
997,Year of Wonders,998,4,3.20
998,You Suck (A Love Story #2),999,2,4.50


Получили среднюю оценку и количество обзоров. При выводе отсортированной таблицы видим, что максимальное количество обзоров - 7 на книгу Twilight (Twilight #1), рейтинг книги почти 3.7.

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц, так мы исключим из анализа брошюры

In [11]:
book_publisher = '''
SELECT
     publishers.publisher AS name_publisher,
     COUNT(books.book_id) AS cnt_of_published_books
FROM
    publishers
INNER JOIN books ON books.publisher_id = publishers.publisher_id
WHERE
    books.num_pages > 50
GROUP BY
    name_publisher
ORDER BY 
    cnt_of_published_books DESC
LIMIT 1
'''
display(book_publisher)

,name_publisher,cnt_of_published_books
0,Penguin Books,42


Издательство Penguin Books выпустило больше всего книг объемом толще 50 страниц.

### Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [12]:
best_author_ever = '''
SELECT
      SUBQ.author AS author,
      AVG(SUBQ.avg_rating) AS avg_rating
FROM
(SELECT
     authors.author AS author,
     books.book_id AS id,
     AVG(ratings.rating) AS avg_rating,
     COUNT(ratings.rating) AS cnt_rating
FROM
    authors
INNER JOIN books ON books.author_id = authors.author_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
    author,
    id
    HAVING
    COUNT(ratings.rating) >= 50) AS SUBQ
GROUP BY
    author
ORDER BY
    avg_rating DESC
LIMIT 1
'''
display(best_author_ever)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.28


Наивысшая средняя оценка у J.K. Rowling/Mary GrandPré. Средняя оценка книг - 4.28.

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [13]:
avg_reviews = '''
SELECT 
    round(avg(count)) AS rounded_avg_review    
FROM
    (SELECT
        COUNT(text)
    FROM 
        reviews
    FULL OUTER JOIN (SELECT username, COUNT(rating_id) as cnt FROM ratings GROUP BY username) AS cnt_rating
    ON cnt_rating.username = reviews.username where cnt > 50
    GROUP BY reviews.username) as count;                                  
'''
display(avg_reviews)

,rounded_avg_review
0,24.00


В среднем 24 обзора от пользователей, которые поставили больше 50 оценок.

## Общий вывод

* После 1 января 2000 года было выпущено 819 книг.
* Получили количество обзоров и среднюю оценку. Наибольшее количество обзоров было сделано на книгу Twiligh (Twilight #1) - 7, рейтинг книги - 3.7.
* Издательство, выпустившее наибольшее число книг толще 50 страниц: Penguin Books, они выпустили 42 книги.
* Cамая высокая средняя оценка у J.K. Rowling/Mary GrandPré (составляет 4.28).
* Среднее количество обзоров пользователей 24, которые поставили более чем по 50 оценок.